# Evaluating Website Redesign Impact with A/B Testing


## 1. Business Context and Problem Statement


In this project, a controlled A/B experiment is evaluated to determine whether a new website landing page leads to a meaningful improvement in user conversion compared to the existing design. Users are randomly assigned to either a control group (old landing page) or a treatment group (new landing page), and conversion outcomes are observed. 

The objective of this analysis is to assess whether the observed difference in conversion performance between the two groups is statistically significant and practically meaningful, and to provide a data-driven recommendation on whether the redesigned page should be adopted.

## 2. Dataset Overview

This analysis is based on user-level data from a controlled A/B experiment designed to evaluate the impact of a website landing page redesign on conversion behavior. Each observation represents an individual user interaction recorded during the experiment.

The dataset is structured to allow a comparison between a control group and a treatment group, with conversion outcome as the primary metric of interest. A detailed inspection of the dataset structure and variables is conducted after loading the data.

### 2.1 Dataset Source

The dataset used in this project was obtained from Kaggle:

- **Source**: Kaggle — A/B Testing Dataset  
- **Link**: https://www.kaggle.com/datasets/zhangluyuan/ab-testing